# Shortest Distance to a Coastline
> A visual guide using geopandas, shapely and folium

I was given the data of a Bicycle company, on analysing it I found out people on the coastlines are more likely to buy a bicycle than people who live in the interior. So I tried to find out the shortest distance to the coastline from each point in the dataset.

This would give me a really nice feature to add to my model.

Read more about the case study [here](https://www.kaggle.com/code/notcostheta/kpmg-virtual-internship-task1)

In [1]:
# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing the Dataset
%pip install shapely==2.0.2
%pip install pandas==2.1.1

import pandas as pd
import numpy as np

GeoCustomers = pd.read_csv('/kaggle/input/geocustomers/GeoCustomers.csv')
GeoCustomers.head()

  Obtaining dependency information for shapely==2.0.2 from https://files.pythonhosted.org/packages/99/e9/a996a080d8478f4ab5ea82f64a5f39aaa8e05c99c2703e0ee03ec8c9e924/shapely-2.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.0 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.5.post1
    Uninstalling Shapely-1.8.5.post1:
      Successfully uninstalled Shapely-1.8.5.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for pandas==2.1.1 from https://files.pythonhosted.org/packages/2f/0e/3b74e8f7c908082793adafb02753477f653ccd7e189f3ba070757d2d0e65/panda

,customer_id,lat,long,state,cluster,customer_category,geometry,nearest_coastline,distance_to_coastline
0,1,-33.894912,151.206211,NSW,2,Gold,POINT (151.206211 -33.894912),POINT (143.87669372558594 -9.145400047302246),25.812016
1,2,-33.731651,150.955942,NSW,3,Bronze,POINT (150.955942 -33.731651),POINT (143.87669372558594 -9.145400047302246),25.585142
2,4,-28.035453,153.241258,QLD,3,Bronze,POINT (153.241258 -28.035453),POINT (143.87669372558594 -9.145400047302246),21.083860
3,5,-30.604667,152.956681,NSW,1,Silver,POINT (152.956681 -30.604667),POINT (143.87669372558594 -9.145400047302246),23.301208
4,6,-38.215906,144.334005,VIC,1,Silver,POINT (144.334005 -38.215906),POINT (143.8740997314453 -9.145500183105469),29.074044


In [3]:
# Create a new dataframe with only the columns we need

df = GeoCustomers[['customer_id', 'lat', 'long']]
df.head()

,customer_id,lat,long
0,1,-33.894912,151.206211
1,2,-33.731651,150.955942
2,4,-28.035453,153.241258
3,5,-30.604667,152.956681
4,6,-38.215906,144.334005


# Getting the world map ready

In [4]:
# Getting australia map from geopandas
import geopandas as gpd

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
country = world[world.name == "Australia"]

# Plotting using folium

import folium
m_1 = folium.Map(location=[-25.2744, 133.7751], tiles='cartodbpositron', zoom_start=4)
folium.GeoJson(country).add_to(m_1)
m_1

In [5]:
# Plotting only the coastline

coastline = country.boundary
m_2 = folium.Map(location=[-25.2744, 133.7751], tiles='cartodbpositron', zoom_start=4)
folium.GeoJson(coastline).add_to(m_2)
m_2

# Explaining the Implementation
- use shapely to create a point from the lat and long
- use shapely.nearst_points to find the nearest point on the coastline
- calculate the distance between the two points using harvesine distance

In [6]:
# Example of how we it actually works
from shapely.ops import Point, nearest_points

point = Point(df["long"][0], df["lat"][0])  # longitude, latitude of base point
nearest = nearest_points(point, coastline)[1]  # nearest point on boundary

m_3 = folium.Map(location=[-33.934109, 151.264326], tiles="cartodbpositron", zoom_start=10)
folium.Marker([df["lat"][0], df["long"][0]]).add_to(m_3)
folium.Marker([nearest.y, nearest.x]).add_to(m_3)
folium.GeoJson(coastline).add_to(m_3)
m_3

Works perfectly, now we will calculate the distance for each point in the dataset

In [7]:
# Calculating the distance between two points in km

import haversine as hs
from haversine import Unit

geo = Point(df['long'][0], df['lat'][0])
cos = Point(nearest.x, nearest.y)

# Calculate the distance using haveversine formula
loc1 = (geo.y, geo.x)
loc2 = (cos.y, cos.x)

result = hs.haversine(loc1, loc2, unit=Unit.KILOMETERS)
print(result)

6.910552542655987


# Writing the function

In [8]:
from shapely.ops import nearest_points
import haversine as hs
from haversine import Unit


def coastline_minima(df, lat_col, long_col, country):
    """
    This function takes in a dataframe with latitude and longitude columns and returns the nearest point on the coastline and the distance in km

    Required libraries: geopandas, shapely, folium, haversine
    Returns: original dataframe with two new columns: nearest_point and distance_to_coast

    """
    # Getting the coastline
    world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    coastline = world[world.name.str.contains(country)].boundary

    # Creating empty lists to store the nearest point and the distance
    nearest_points_list = []
    distance_list = []

    for i in range(len(df)):
        geo = Point(df[long_col][i], df[lat_col][i])
        nearest = nearest_points(geo, coastline)[1]
        cos = Point(nearest.x, nearest.y)
        loc1 = (geo.y, geo.x)
        loc2 = (cos.y, cos.x)
        result = hs.haversine(loc1, loc2, unit=Unit.KILOMETERS)

        nearest_points_list.append(nearest)
        distance_list.append(result)

    # Adding the new columns to the dataframe
    df["nearest_point"] = nearest_points_list
    df["distance_to_coast"] = distance_list

    return df

In [9]:
coastline_minima(df, 'lat', 'long', 'Australia')

,customer_id,lat,long,nearest_point,distance_to_coast
0,1,-33.894912,151.206211,137 POINT (151.26433 -33.93411) dtype: geom...,6.910553
1,2,-33.731651,150.955942,137 POINT (151.26176 -33.93792) dtype: geom...,36.385439
2,4,-28.035453,153.241258,137 POINT (153.45911 -27.91328) dtype: geom...,25.342373
3,5,-30.604667,152.956681,137 POINT (153.07812 -30.60035) dtype: geom...,11.632766
4,6,-38.215906,144.334005,137 POINT (144.33147 -38.21284) dtype: geom...,0.406144
...,...,...,...,...,...
3407,3496,-33.937716,150.848737,137 POINT (151.13271 -34.12925) dtype: geom...,33.739183
3408,3497,-38.043995,145.264296,137 POINT (145.01069 -37.96847) dtype: geom...,23.754216
3409,3498,-37.807135,144.861162,137 POINT (144.90698 -37.93953) dtype: geom...,15.260753
3410,3499,-27.549179,152.951385,137 POINT (153.18229 -27.41968) dtype: geom...,26.947659


Works perfectly fine if you ignore the warnings xD


# References
- geopandas: [link](https://geopandas.org/)
- shapely: [link](https://shapely.readthedocs.io/)
- folium: [link](https://python-visualization.github.io/folium/)
- haversine: [link](https://pypi.org/project/haversine/)

# A few words from the author
Well actually I'm really embarresed to say that it took a whole day for me to figure out what to do, the trick was to get the coordanates into points in one way or another using shapely, rest was a piece of cake.

There were three iterations of this notebook on my local machine, the geospacial analysis space really needs a more userfriendly alternative for real.

It is also very visible that I have avoided the use of shape files in here, that's because I did not succeed in the other two attempts and it was the end of my logic and willpower.

Also the shapefile dealt with very high resolution and took lakes and river bodies into account, which messed up with the coastline logic and gave me some funny results. The .border method really came to my rescue to be honest.

Anyways if any of this helped you, please consider upvoting or follow me maybe, my case study will be out soon.
